<a href="https://colab.research.google.com/github/lfj95/SENSE2023/blob/main/training_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Semantic segmentation of Aerial Imagery with U-Net-like architectures**


### This notebook is a simple semantic segmentation program. The goal is to build and train a model which is able to automatically label each pixel in an aerial image with its semantic category. 

### The dataset we use consists of aerial imagery of Dubai obtained by MBRSC satellites and annotated with pixel-wise semantic segmentation in 6 classes. The total volume of the dataset is 1305 256*256 images grouped into 6 larger tiles. The classes are: Building, Land (unpaved area), Road, Vegetation, Water, Unlabeled.

### Run on GPU

For model runtime speed, check Collab is running on GPU. 

Click the “**Runtime**” dropdown menu. Select “**Change runtime type**”. Now select '**GPU**' in the “**Hardware accelerator**” dropdown menu. 


### Install Python library
```segmentation-models```: with Neural Networks for Image Segmentation based on Keras and TensorFlow.

In [ ]:
!pip install -U segmentation-models

### Download Dataset from GitHub
```git-clone``` - Clone a repository into a new directory

In [ ]:
!git clone https://github.com/lfj95/SENSE2023.git

### Import Packages

In [ ]:
import os
import cv2
import numpy as np
import random
import pickle
from keras.models import Model
from keras.layers import *
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import plot_model
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm


### Set the Current Working Directory

In [9]:
def makedir(new_dir):
  if not os.path.exists(new_dir):
    os.makedirs(new_dir)

label_directory = os.getcwd() + "/SENSE2023/split_data/labels.npy"
train_data_directory = os.getcwd() + "/SENSE2023/split_data/train/images"
valid_data_directory = os.getcwd() + "/SENSE2023/split_data/valid/images"
test_data_directory = os.getcwd() + "/SENSE2023/split_data/test/images"
save_directory = os.getcwd() + "/models"
makedir(save_directory)

### Read train/ validation/ test images from corresponding subdirectory.

In [ ]:
def read_image(path):
  image_dataset = []
  images = os.listdir(path)  #List of all image names in this subdirectory
  images = sorted(images)
  for i, image_name in enumerate(images):  
      if image_name.endswith(".jpg"):   #Only read jpg images...
          image = cv2.imread(path+"/"+image_name, 1)  #Read each image as BGR
          image_dataset.append(image)
  return image_dataset

X_train = np.array(read_image(train_data_directory))
X_valid = np.array(read_image(valid_data_directory))
X_test = np.array(read_image(test_data_directory))
print("Train Images size:", X_train.shape, '\n', "Validation Images size:",X_valid.shape, '\n', "Test Images size:",X_test.shape )

### Read array (integer values for each pixel) from .npy file,  to be used as labels. 
```
tf.keras.utils.**to_categorical**(y, num_classes=None, dtype="float32")
```
Converts a class vector (integers) to binary class matrix, e.g., for use with categorical_crossentropy.

In [11]:
dict_load=np.load(label_directory, allow_pickle=True)

labels_train = dict_load.item()['labels_train']
labels_valid = dict_load.item()['labels_valid']
labels_test = dict_load.item()['labels_test']
n_classes = len(np.unique(labels_train))

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(labels_train, num_classes=n_classes)
y_valid = to_categorical(labels_valid, num_classes=n_classes)
y_test = to_categorical(labels_test, num_classes=n_classes)


### Define the model: U-Net based model

The model we defined consists of a contracting path and an expansive path. 
*   The contracting path follows the typical architecture of a convolutional network. It consists of the repeated application of two 3x3 convolutions, each followed by a rectified linear unit (ReLU) and a 2x2 max pooling operation with stride 2 for downsampling. 
*   At each downsampling step we double the number of feature channels. Every step in the expansive path consists of an upsampling of the feature map followed by a 2x2 convolution (“up-convolution”) that halves the number of feature channels, a concatenation with the correspondingly cropped feature map from the contracting path, and two 3x3 convolutions, each followed by a ReLU. 

At the final layer, a 1x1 convolution is used to map each 64-component feature vector to the desired number of classes. 

Layers used: 

1.   [Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/) 
2.   [MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) 
3.  [UpSampling2D](https://keras.io/api/layers/reshaping_layers/up_sampling2d/) 









In [12]:
#  unet mini

num_classes=6
IMG_HEIGHT=256
IMG_WIDTH=256
IMG_CHANNELS=3

inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

# contracting path
c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
c1 = Conv2D(16, 3, activation='relu', padding='same')(c1)
p1 = MaxPooling2D(pool_size=(2, 2))(c1)

c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
c2 = Conv2D(32, 3, activation='relu', padding='same')(c2)
p2 = MaxPooling2D(pool_size=(2, 2))(c2)  
  
c3 = Conv2D(64, 3, activation='relu', padding='same')(p2)
c3 = Conv2D(64, 3, activation='relu', padding='same')(c3)

# expansive path
c4 = UpSampling2D(size=(2, 2))(c3)
c4 = concatenate([c4, c2])
c4 = Conv2D(32, 3, activation='relu', padding='same')(c4)
c4 = Conv2D(32, 3, activation='relu', padding='same')(c4)
  
c5 = UpSampling2D(size=(2, 2))(c4)
c5 = concatenate([c5, c1])
c5 = Conv2D(16, 3, activation='relu', padding='same')(c5)
c5 = Conv2D(16, 3, activation='relu', padding='same')(c5)

outputs = Conv2D(num_classes, 1, activation='softmax')(c5)

model = keras.Model(inputs, outputs)

# model.summary()


### Model plotting utilities
```
tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=False, show_dtype=False, show_layer_names=True, rankdir="TB", expand_nested=False, dpi=96, layer_range=None, show_layer_activations=False)
```
Converts a Keras model to dot format and save to a file.

In [ ]:
plot_model(model, to_file= save_directory + '/small_unet1.png', show_shapes=True)

### Configures the model for training.
```
compile(optimizer='rmsprop', loss=None, metrics=None, loss_weights=None, weighted_metrics=None, run_eagerly=None, steps_per_execution=None,jit_compile=None, **kwargs)
```


> *   optimizer:	String (name of optimizer) or optimizer instance. See tf.keras.
optimizers.
*   loss: 	Loss function. Maybe be a string (name of loss function), or a tf.keras.losses.Loss instance. See tf.keras.losses.
*   metrics: List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 







In [14]:
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)
metrics=['accuracy', jacard_coef]

model.compile(optimizer='adam', loss=sm.losses.CategoricalCELoss(), metrics=metrics)
# model.summary()

### Train the model for a fixed number of epochs (iterations on a dataset).

```
model.fit(x=None, y=None, batch_size=None, epochs=1, verbose="auto", callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_batch_size=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False,)
```


> *    x: Input data. It could be either Numpy array(s) or TensorFlow tensor(s).
*    y: Target data. Like the input data x, it could be either Numpy array(s) or TensorFlow tensor(s). It should be consistent with x.
*    batch_size: Integer or None. Number of samples per gradient update. If unspecified, batch_size will default to 32.
*    epochs: Integer. Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
*    validation_data: Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data.
*    shuffle: Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').




In [ ]:
history1 = model.fit(X_train, y_train,
                    batch_size = 16, 
                    verbose=1, 
                    epochs=60, 
                    validation_data=(X_valid, y_valid), 
                    shuffle=True)

In [ ]:
model.save(save_directory + '/models/small_unet1.hdf5')

### Plot the training and validation accuracy and loss at each epoch

In [ ]:
history = history1
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['jacard_coef']
val_acc = history.history['val_jacard_coef']

plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.show()


### Visualize predictions

In [ ]:
test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth = np.argmax(y_test, axis = 3)[test_img_number]
#test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img)
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth, vmin=0, vmax=5)
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img, vmin=0, vmax=5)
plt.show()

### Exercises 1. Finetune the segmentation model 

Now try adjusting the model by implementing different methods of reducing overfitting and increasing the model's ability to generalise.









1.   Retrain the model by using or adding a different loss function and see what happens to the performance of the segmentation model. The segmentation models losses can be combined together by '+' and scaled by integer or float factor set class weights for dice_loss.
E.g., add in a **CategoricalFocalLoss**/**dice_loss** in the Configure the model for training section.

 >Hint: Further details for loss options can be found in [segmentation losses](https://segmentation-models.readthedocs.io/en/latest/api.html?highlight=loss#losses)







 









2.   In the previous model, the contracting path follows the formula: 

  `conv_layer1 -> conv_layer2 -> max_pooling` 
  
  In the expansive path, the image is going to be upsized to its original size. The formula follows:

  `UpSampling2D -> concatenate -> conv_layer1 -> conv_layer2` 

  We built 2 contracting blocks and 2 expanding blocks. Could you try lessening the block or repeating the block more times? Does this make the segmentation results worse or better?
  Beside, you can also explore removing one of the two convolutions in every contracting/expansive block.

  > Hint: Make sure the start and end image dimensions match, check the UNET structure in you’ve created small_unet1.png




3.   Change the CONV layers with Batch Normalization & Dropout. 


*   One common issue in training a Segmentation model based on Deep Convolutional Neural Network for is **the internal covariate shift**, where the training of convolutional kernels is encumbered by the distribution change of input features, hence both the training speed and performance are decreased. Batch Normalization is the first proposed method for addressing internal
covariate shift and is widely used. 
*   Since **parameter size** in some Conv layers are quite large, and the training data is small. It is reasonable to add dropout between larg convolutional connections. 

  > Hint: 
  *   Keras Implementation of [Batch normalisation](https://keras.io/api/layers/normalization_layers/batch_normalization/)
  *   Keras Implementation of [Dropout](https://keras.io/api/layers/regularization_layers/dropout/)



4.   Data augmentation of your choice. 

  > Hint: **scipy.ndimage** module provides a powerful set of general, n-dimensional image processing operations. [Interpolation in scipy.ndimage](https://docs.scipy.org/doc/scipy-0.14.0/reference/ndimage.html#module-scipy.ndimage.interpolation) include basic operations like shifts, rotations and resizing as well as more general coordinate transformations like affine transforms or warping. See the following rotate example: 

  ```
scipy.ndimage.interpolation.rotate(input, angle, axes=(1, 0), reshape=True, output=None, order=3, mode='constant', cval=0.0, prefilter=True)
```




### Exercises 2.Try another model
Change the network architecture to U-net with Resnet backbone. Use pretrainded weights to initialize it.

> Hint: To do this, follow the instructions in [Segmentation Models Python API](https://segmentation-models.readthedocs.io/en/latest/tutorial.html?highlight=loss#simple-training-pipeline), and you can implete U-net with pretrained weights in simple two lines. It provides 4 models architectures for binary and multi class segmentation (including legendary Unet) and 25 available backbones for each architecture.  All backbones have pre-trained weights for faster and better convergence.



